# Titanic random-forest classifier - Pipeline

# Install packages

In [11]:
#install as trusted host
#!pip install --index-url=http://pypi.python.org/simple/ --trusted-host pypi.python.org lime
#!pip install --index-url=http://pypi.python.org/simple/ --trusted-host pypi.python.org treeinterpreter
#!pip install --index-url=http://pypi.python.org/simple/ --trusted-host pypi.python.org graphviz
#!pip install --index-url=http://pypi.python.org/simple/ --trusted-host pypi.python.org pydotplus

# Import libraries

In [3]:
# Import libraries
import numpy as np
import pandas as pd
import pylab as pl
import datetime

from treeinterpreter import treeinterpreter as ti

import lime
import lime.lime_tabular

from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# include plots inline in the notebook (ipython specific)
%matplotlib inline

# Input cell

In [4]:
path = 'titanic-train.csv' #Enter data path file here
keep = ['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'] #Keep needed features only

one_hot_encoding = True #one-hot encoding for categorical values
dropna = True #Do you want to drop missing values?
impute = False #Do you want to impute missing values?
missing_values = 'NaN' #Make sure missing data has been replaced with NaN first
impute_strategy = 'most_frequent' # 'most_frequent', 'mean', 'median'
scale_features = False #Do you want to scale numerical figures?

target = 'Survived' #Target variable

seed = 42 #For random state
test_size = 0.4 #For train_test_split

# Read data set

In [5]:
# Read datasets from excel files
df = pd.read_csv(path)
print("Dataset has {} rows, {} columns".format(*df.shape))
df.head(5)

Dataset has 891 rows, 12 columns


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Data pre-processing

In [6]:
#### Standardisable pre-processing

#Keep needed features only
df = df[keep]

#Dummy encoding
if one_hot_encoding:
    df = pd.get_dummies(df, drop_first=True)

#Drop missing values
if dropna:
    df = df.dropna(how='any') #'any', 'all'
    
if impute:
    imp = Imputer(missing_values=missing_values, strategy=impute_strategy, axis=0)
    imp.fit_transform(df)
    
#Scaling of numerical data
if scale_features:
    scaler = StandardScaler()
    scaler.fit_transform(df)
    
print("Dataset has {} rows, {} columns".format(*df.shape))

Dataset has 714 rows, 9 columns


In [7]:
#Strings needs to be converted to integers:
# 1. Name - This can be removed
# 2. Sex - Converted into 1/0
# 3. Ticket - Probably insignificant, remove for now.
# 4. Cabin - Might represent where they are in the ship, remove for now.
# 5. Embarked - 'Matrix conversion' as no levels inherent in number

#Remove floats - value too large for random forest
df['Age'] = df['Age'].astype(int)
df['Fare'] = df['Fare'].astype(int)

In [8]:
#Create response and target variable
X = df.drop(target, axis=1)
y = df[target]

# Machine Learning
Here we train and fit the model.

In [9]:
#Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

## Decision tree classifier

In [10]:
#Train model
dt = DecisionTreeClassifier(max_depth=3)
dt.fit(X_train, y_train)

# Make prediction
dt_y_pred = dt.predict(X_test)
dt_y_pred_proba = dt.predict_proba(X_test)

# Compute and print metrics
print("Accuracy: {}".format(dt.score(X_test, y_test)))
print(classification_report(y_test, dt_y_pred))

Accuracy: 0.7692307692307693
             precision    recall  f1-score   support

          0       0.77      0.85      0.81       165
          1       0.77      0.65      0.71       121

avg / total       0.77      0.77      0.77       286



In [11]:
#Building the dot file for visualisation

from sklearn import tree
import pydotplus

clf = dt
clf = clf.fit(X_train, y_train)
tree.export_graphviz(clf, out_file = 'tree_max_depth_3.dot', 
                    feature_names=X_test.columns.values, class_names=['Died','Survived'],
                    filled=True, rounded=True, special_characters=True)

In [13]:
#Display in Jupyter

from sklearn.externals.six import StringIO
from IPython.display import Image 
import graphviz

dot_data = StringIO()
tree.export_graphviz(clf, out_file = dot_data, 
                     feature_names=X_test.columns.values, class_names=['Did not survive', 'Survived'],
                    filled=True, rounded=True, special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())

#GraphViz executables not found:  graph.write_png('tree_max_depth_3.png')

## Random forest classifier

In [14]:
#Tuning the model
param_grid = { "n_estimators"      : [100, 150],
           "criterion"         : ["gini"],
           "max_features"      : ['auto'], #auto, sqrt, log2, int/n_feature
           "max_depth"         : [5, 20],
           "min_samples_split" : [2, 4] ,
           "bootstrap": [True]}

rf = RandomForestClassifier(random_state=seed)

rf_cv = GridSearchCV(rf, param_grid, cv=5)
rf_cv.fit(X_train, y_train)

rf_best = rf_cv.best_estimator_

# Make prediction
y_pred = rf_cv.predict(X_test)
y_pred_proba = rf_cv.predict_proba(X_test)

# Compute and print metrics
print("Accuracy: {}".format(rf_cv.score(X_test, y_test)))
print("Best score is {}".format(rf_cv.best_score_))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(rf_cv.best_params_))

Accuracy: 0.7762237762237763
Best score is 0.8481308411214953
             precision    recall  f1-score   support

          0       0.75      0.91      0.82       165
          1       0.83      0.60      0.69       121

avg / total       0.78      0.78      0.77       286

Tuned Model Parameters: {'bootstrap': True, 'criterion': 'gini', 'max_depth': 5, 'max_features': 'auto', 'min_samples_split': 2, 'n_estimators': 100}


## Feature importance
What are the most important drivers of store sales according to the model

In [15]:
#Display feature importance
def feature_importance(model, trainData, display_n_rows):
    """Display feature importance & weighting for tree based model"""
    fi = model.feature_importances_*100
    feat_imp = pd.DataFrame(list(zip(fi,trainData.columns.values)))
    feat_imp = feat_imp.sort_values(by=0, axis=0, ascending=False)
    feat_imp.columns = ['importance %', 'feature']
    print(feat_imp[:display_n_rows])

In [16]:
#Display features & weighting
feature_importance(rf_best, X_train, 10)

   importance %     feature
5     41.920456    Sex_male
1     16.694158         Age
4     16.309330        Fare
0     14.161379      Pclass
3      4.039234       Parch
2      3.621181       SibSp
7      2.631645  Embarked_S
6      0.622617  Embarked_Q


## Decision path visualisation

In [17]:
i_data = X_test.iloc[12]
data = i_data.values.reshape(1,-1)
d_path = rf_best.decision_path(data)
print(d_path)

(<1x3982 sparse matrix of type '<class 'numpy.int64'>'
	with 598 stored elements in Compressed Sparse Row format>, array([   0,   45,   98,  149,  190,  233,  258,  309,  360,  401,  430,
        461,  512,  541,  580,  623,  668,  711,  760,  803,  852,  889,
        932,  981, 1006, 1035, 1074, 1107, 1136, 1165, 1196, 1241, 1262,
       1313, 1350, 1385, 1420, 1465, 1518, 1553, 1590, 1625, 1672, 1707,
       1744, 1787, 1812, 1863, 1904, 1945, 1982, 2017, 2054, 2097, 2142,
       2191, 2228, 2267, 2304, 2343, 2390, 2419, 2456, 2489, 2534, 2583,
       2632, 2677, 2714, 2739, 2786, 2833, 2886, 2919, 2960, 2995, 3032,
       3073, 3126, 3157, 3194, 3239, 3274, 3313, 3354, 3409, 3458, 3483,
       3516, 3539, 3590, 3629, 3660, 3707, 3750, 3777, 3822, 3861, 3898,
       3939, 3982], dtype=int32))


## Tree interpreter
Decomposing random forest predictions

In [18]:
#Print for all instances 

#Run tree interpreter
prediction, bias, contributions = ti.predict(rf_best, X_test)

#Create df
survived_contributions = contributions[:, :, 1]
df_contributions = pd.DataFrame(survived_contributions, columns=X.columns)
df_contributions['ti_bias'] = bias[:,1]
df_contributions['rf_survived_proba'] = y_pred_proba[:,1]
df_contributions['predicted_outcome'] = y_pred
df_contributions['actual_outcome'] = y_test
df_contributions['actual_outcome'].fillna(0, inplace=True)
df_contributions = df_contributions.round(4) * 100
df_contributions[['predicted_outcome', 'actual_outcome']] = df_contributions[['predicted_outcome', 'actual_outcome']] / 100
df_contributions.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,ti_bias,rf_survived_proba,predicted_outcome,actual_outcome
0,-2.17,-2.25,-0.24,-1.07,-3.09,-16.03,-0.02,-1.31,39.67,13.49,0.0,0.0
1,2.91,33.94,3.56,4.71,4.70,-14.97,0.08,-1.12,39.67,73.47,1.0,0.0
2,7.88,0.64,-0.76,-0.23,0.59,30.32,-0.05,-1.51,39.67,76.53,1.0,1.0
3,9.29,1.96,0.33,0.26,9.83,28.04,0.21,3.84,39.67,93.43,1.0,0.0
4,-5.32,-3.36,0.12,-0.72,-7.01,-11.95,-0.19,-1.37,39.67,9.86,0.0,0.0


In [19]:
#Create function to show instance alongside contributions

def instance(i):
    """Actual test example alongside contributions"""
    i_contribution = df_contributions.iloc[i]
    i_data = X_test.iloc[i]
    i_actual = y_test.iloc[i]

    instance = pd.DataFrame(list(zip(X_test.columns.values, i_data, i_contribution)), 
                            columns = ['Feature', 'Instance', 'Contribution'])
    print("Instance {} prediction:".format(i), rf_best.predict(i_data.values.reshape(1,-1)))
    print("Instance {} actual:".format(i), i_actual)
    print(instance)

## Outputs
- Which features are driving each outcome?
- What does each outcome look like?

In [20]:
#Write contributions to csv
df_contributions.to_csv('contributions.csv')

In [21]:
#Output for example instance
instance(2)

Instance 2 prediction: [1]
Instance 2 actual: 1
      Feature  Instance  Contribution
0      Pclass         2          7.88
1         Age        29          0.64
2       SibSp         1         -0.76
3       Parch         0         -0.23
4        Fare        26          0.59
5    Sex_male         0         30.32
6  Embarked_Q         0         -0.05
7  Embarked_S         1         -1.51


In [22]:
df_contributions.groupby('predicted_outcome').mean().round(2).drop('actual_outcome', axis=1)

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,ti_bias,rf_survived_proba
predicted_outcome,,,,,,,,,,
0.0,-3.30,-0.46,-1.0,-0.33,-2.55,-8.78,-0.07,-0.45,39.67,22.73
1.0,4.71,2.45,0.9,1.31,1.98,25.24,0.04,1.51,39.67,77.81


In [23]:
#Display features & weighting
feature_importance(rf_best, X_train, 10)

   importance %     feature
5     41.920456    Sex_male
1     16.694158         Age
4     16.309330        Fare
0     14.161379      Pclass
3      4.039234       Parch
2      3.621181       SibSp
7      2.631645  Embarked_S
6      0.622617  Embarked_Q


## Bayesian Rule List classifier
- Produce rule lists

Questions for Aldous:

1. Understand the use of 'class' object
2. How to interpret 'contributions'?